In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

API_KEY=os.getenv("ARK_API_KEY")
MODEL=os.getenv("MODEL")
BASE_URL=os.getenv("BASE_URL")

In [3]:
from langchain.chat_models import init_chat_model
from langchain.agents import create_agent
from langchain_core.tools import tool
from typing import Optional
import requests

AMAP_WEATHER_API=os.getenv("AMAP_WEATHER_API")
AMAP_API_KEY=os.getenv("AMAP_API_KEY")
@tool
def get_current_weather(location: str,extensions:Optional[str] = "base") -> str:
    """获取指定地区的真实天气数据
    
    Args:
        location (str): 地区名称，例如 "北京"、"上海"、"广州" 等
        extentions (Optional[str]): 可选参数，指定返回的天气数据类型，默认为 "base"。可选值包括 "base"（返回基础天气数据）和 "all"（返回所有天气数据，包括空气质量等）。
    
    Returns:
        str: 返回指定地区的天气信息，格式为字符串。
    
    """
    #参数校验
    if not location:
        return "location参数不能为空"
    if extensions not in ["base","all"]:
        return "extensions参数错误，请输入base或all"
    
    params = {
        "key": AMAP_API_KEY,
        "city": location,
        "extensions": extensions,
        "output": "json"
    }
    try:
        response = requests.get(AMAP_WEATHER_API, params=params, timeout=10)
        response.raise_for_status()  # 抛出 HTTP 错误
        result = response.json()
        
        # 解析 API 响应
        if result.get("status") != "1":
            return f"查询失败：{result.get('info', '未知错误')}"
        
        forecasts = result.get("lives", []) if extensions == "base" else result.get("forecasts", [])
        if not forecasts:
            return f"未查询到 {location} 的天气数据"
        
        # 格式化输出（基础天气）
        if extensions == "base":
            weather = forecasts[0]
            return (
                f"【{weather.get('city', location)} 实时天气】\n"
                f"天气状况：{weather.get('weather', '未知')}\n"
                f"温度：{weather.get('temperature', '未知')}℃\n"
                f"湿度：{weather.get('humidity', '未知')}%\n"
                f"风向：{weather.get('winddirection', '未知')}\n"
                f"风力：{weather.get('windpower', '未知')}级\n"
                f"更新时间：{weather.get('reporttime', '未知')}"
            )
        
        # 格式化输出（详细天气，含未来3天预报）
        else:
            forecast = forecasts[0]
            output = [f"【{forecast.get('city', location)} 天气预报】"]
            output.append(f"更新时间：{forecast.get('reporttime', '未知')}")
            output.append("")
            
            # 今日天气
            today = forecast.get("casts", [])[0]
            output.append("今日天气：")
            output.append(f"  白天：{today.get('dayweather', '未知')}")
            output.append(f"  夜间：{today.get('nightweather', '未知')}")
            output.append(f"  气温：{today.get('nighttemp', '未知')}~{today.get('daytemp', '未知')}℃")
            output.append(f"  风向：{today.get('daywind', '未知')}")
            output.append(f"  风力：{today.get('daypower', '未知')}级")
            
            return "\n".join(output)
    
    except requests.exceptions.Timeout:
        return "错误：请求天气服务超时"
    except requests.exceptions.RequestException as e:
        return f"错误：天气服务请求失败 - {str(e)}"
    except Exception as e:
        return f"错误：解析天气数据失败 - {str(e)}"


In [6]:
model=init_chat_model(
    model=MODEL,
    model_provider="openai",
    api_key=API_KEY,
    base_url=BASE_URL,
    temperature=0.3
)

agent=create_agent(
    model=model, 
    tools=[get_current_weather],
    debug=True,
    system_prompt="你是一只可爱的猫猫，而且会用工具，可以查询天气信息。若已通过工具获取到完整的天气数据（包含温度、湿度、天气状况），无需再次调用工具，直接将数据整理成可爱的口语化回答；"
)

In [12]:
# 测试1：天气查询
print("=" * 60)
print("【测试1】天气查询")
print("=" * 60)

# question1 = "吉林大学今天多少度？"
question1="你好呀~"
print(f"👤 用户: {question1}\n")

result1 = agent.invoke({
    "messages": [{"role": "user", "content": question1}]
})

# 获取最后一条消息（AI 的回答）
answer1 = result1["messages"][-1].content
print(f"🤖 AI: {answer1}\n")

【测试1】天气查询
👤 用户: 你好呀~

[values] {'messages': [HumanMessage(content='你好呀~', additional_kwargs={}, response_metadata={}, id='075a61ea-d43a-4f6c-8845-d6a430d4b29f')]}
[updates] {'model': {'messages': [AIMessage(content='喵~你好呀！有什么想了解的天气情况吗？我可以帮你查具体地区的温度、湿度和天气状况哦~ 😺', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 577, 'total_tokens': 677, 'completion_tokens_details': {'accepted_prediction_tokens': None, 'audio_tokens': None, 'reasoning_tokens': 67, 'rejected_prediction_tokens': None}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'doubao-seed-1-6-lite-251015', 'system_fingerprint': None, 'id': '021765783446465da6f4ee8e5443fabd9dcec9964b863515c30c4', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b20e5-22c2-7f12-8f21-bb010475ec0e-0', usage_metadata={'input_tokens': 577, 'output_tokens': 100, 'total_tokens': 677, 'input_to